In [1]:
import sys
from pathlib import Path

current_dir = Path('.')
current_dir = current_dir.absolute()
root_dir = current_dir.parent.parent
src_dir = root_dir / 'src'

sys.path.append(str(src_dir))

%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

load_dotenv(str(src_dir / '.env'))

True

In [3]:
from api.containers import analyze_container

analyze = analyze_container.service()
df = analyze.collect_all()
df

ImportError: cannot import name 'CSVFactory' from 'infra.storage.factories' (F:\Projects\hltv_notebook\src\infra\storage\factories\__init__.py)

In [1]:
# Columns Team1 and Team2 has score as (19). Remove it and add it as a separate column Team1 Score and Team2 Score
df['Team1'] = df['Team1'].str.replace(r'\s\(\d+\)$', '')
df['Team2'] = df['Team2'].str.replace(r'\s\(\d+\)$', '')


NameError: name 'df' is not defined

In [14]:
import re

def get_score(value: str) -> int:
    return int(re.search(r'\((\d+)\)$', value).group(1))

df['Team1 Score'] = list(map(get_score, df['Team1'].tolist()))
df['Team2 Score'] = list(map(get_score, df['Team2'].tolist()))
df

,Date,Team1,Team2,Map,Event,Team1 Score,Team2 Score
0,\n 23/2/23\n,Let us cook (18),9INE (18),\n vtg\n Vertigo\n,European Development Championship 7,18,18
1,\n 23/2/23\n,Eternal Fire (16),Evil Geniuses (14),\n ovp\n Overpass\n,ESL Pro League Season 17,16,14
2,\n 23/2/23\n,Evil Geniuses (6),Eternal Fire (16),\n anb\n Anubis\n,ESL Pro League Season 17,6,16
3,\n 23/2/23\n,Into the Breach (5),The Prodigies (16),\n anb\n Anubis\n,European Pro League Season 6,5,16
4,\n 23/2/23\n,MIBR (16),IHC (12),\n inf\n Inferno\n,ESL Pro League Season 17,16,12
...,...,...,...,...,...,...,...
45,22/2/23,G2 (16),Eternal Fire (9),anb Anubis,ESL Pro League Season 17,16,9
46,22/2/23,Evil Geniuses (4),Cloud9 (16),mrg Mirage,ESL Pro League Season 17,4,16
47,22/2/23,Cloud9 (16),Evil Geniuses (5),inf Inferno,ESL Pro League Season 17,16,5
48,22/2/23,iNation (16),FORZE (12),inf Inferno,CCT West Europe Series 1,16,12


In [15]:
for column in df.columns:
    df[column].replace({r"\\n": ''}, inplace=True, regex=True)
df

,Date,Team1,Team2,Map,Event,Team1 Score,Team2 Score
0,23/2/23,Let us cook (18),9INE (18),vtg Vertigo,European Development Championship 7,18,18
1,23/2/23,Eternal Fire (16),Evil Geniuses (14),ovp Overpass,ESL Pro League Season 17,16,14
2,23/2/23,Evil Geniuses (6),Eternal Fire (16),anb Anubis,ESL Pro League Season 17,6,16
3,23/2/23,Into the Breach (5),The Prodigies (16),anb Anubis,European Pro League Season 6,5,16
4,23/2/23,MIBR (16),IHC (12),inf Inferno,ESL Pro League Season 17,16,12
...,...,...,...,...,...,...,...
45,22/2/23,G2 (16),Eternal Fire (9),anb Anubis,ESL Pro League Season 17,16,9
46,22/2/23,Evil Geniuses (4),Cloud9 (16),mrg Mirage,ESL Pro League Season 17,4,16
47,22/2/23,Cloud9 (16),Evil Geniuses (5),inf Inferno,ESL Pro League Season 17,16,5
48,22/2/23,iNation (16),FORZE (12),inf Inferno,CCT West Europe Series 1,16,12


In [ ]:
# Remove score from team name

def get_team(value: str) -> str:
    return re.search(r'.+?(?=\s\()', value).group()

df['Team1'] = list(map(get_team, df['Team1'].tolist()))
df['Team2'] = list(map(get_team, df['Team2'].tolist()))
df

In [22]:
from typing import Generator, Optional
import itertools


def possible_final_scores() -> Generator[int, None, None]:
    yield from itertools.count(start=16, step=3)


class Score(int):

    def is_final(self) -> bool:
        for score in possible_final_scores():
            if score == self:
                return True

            if score > self:
                break

        return False

    def __contains__(self, score: int) -> bool:
        return self > score

    def __or__(self, other: 'Score') -> Optional['Score']:
        if self > other and self.is_final():
            return self

        elif self < other and other.is_final():
            return other

        return None

In [29]:
sub = df.filter(items=['Team1 Score', 'Team2 Score'])
sub

,Team1 Score,Team2 Score
0,18,18
1,16,14
2,6,16
3,5,16
4,16,12
...,...,...
45,16,9
46,4,16
47,16,5
48,16,12


In [40]:
# Score(df['Team1 Score']) | Score(df['Team2 Score'])
# del sub['Team1 Is Final']
# del sub['Team2 Is Final']
df['Team1 Won'] = df['Team1 Score'].map(Score.is_final)
df['Team2 Won'] = df['Team2 Score'].map(Score.is_final)
# sub['Match Completed'] = sub['Team1 Is Final'] | sub['Team2 Is Final']
# sub['Match Completed'] = sub['Team1 Score'].map(Score.is_final) | sub['Team2 Score'].map(Score.is_final)
# del sub['Match Completed']
df

,Date,Team1,Team2,Map,Event,Team1 Score,Team2 Score,Team1 Won,Team2 Won
0,23/2/23,Let us cook (18),9INE (18),vtg Vertigo,European Development Championship 7,18,18,False,False
1,23/2/23,Eternal Fire (16),Evil Geniuses (14),ovp Overpass,ESL Pro League Season 17,16,14,True,False
2,23/2/23,Evil Geniuses (6),Eternal Fire (16),anb Anubis,ESL Pro League Season 17,6,16,False,True
3,23/2/23,Into the Breach (5),The Prodigies (16),anb Anubis,European Pro League Season 6,5,16,False,True
4,23/2/23,MIBR (16),IHC (12),inf Inferno,ESL Pro League Season 17,16,12,True,False
...,...,...,...,...,...,...,...,...,...
45,22/2/23,G2 (16),Eternal Fire (9),anb Anubis,ESL Pro League Season 17,16,9,True,False
46,22/2/23,Evil Geniuses (4),Cloud9 (16),mrg Mirage,ESL Pro League Season 17,4,16,False,True
47,22/2/23,Cloud9 (16),Evil Geniuses (5),inf Inferno,ESL Pro League Season 17,16,5,True,False
48,22/2/23,iNation (16),FORZE (12),inf Inferno,CCT West Europe Series 1,16,12,True,False


In [42]:
df = df[df['Team1 Won'] | df['Team2 Won']]
df

,Date,Team1,Team2,Map,Event,Team1 Score,Team2 Score,Team1 Won,Team2 Won
1,23/2/23,Eternal Fire (16),Evil Geniuses (14),ovp Overpass,ESL Pro League Season 17,16,14,True,False
2,23/2/23,Evil Geniuses (6),Eternal Fire (16),anb Anubis,ESL Pro League Season 17,6,16,False,True
3,23/2/23,Into the Breach (5),The Prodigies (16),anb Anubis,European Pro League Season 6,5,16,False,True
4,23/2/23,MIBR (16),IHC (12),inf Inferno,ESL Pro League Season 17,16,12,True,False
5,23/2/23,The Prodigies (11),Into the Breach (16),mrg Mirage,European Pro League Season 6,11,16,False,True
...,...,...,...,...,...,...,...,...,...
45,22/2/23,G2 (16),Eternal Fire (9),anb Anubis,ESL Pro League Season 17,16,9,True,False
46,22/2/23,Evil Geniuses (4),Cloud9 (16),mrg Mirage,ESL Pro League Season 17,4,16,False,True
47,22/2/23,Cloud9 (16),Evil Geniuses (5),inf Inferno,ESL Pro League Season 17,16,5,True,False
48,22/2/23,iNation (16),FORZE (12),inf Inferno,CCT West Europe Series 1,16,12,True,False


In [ ]:
teams['Team1 Won'] = teams['Team1 Score'] > teams['Team2 Score']
teams['Team2 Won'] = teams['Team1 Score'] < teams['Team2 Score']
teams

In [ ]:
winners = []

for index, row in teams.iterrows():
    if row['Team1 Won']:
        winners.append(row['Team1'])
    else:
        winners.append(row['Team2'])

teams['Winner'] = winners
teams

In [ ]:
full_teams = teams.copy(deep=True)
teams = teams[['Date', 'Team1', 'Team2', 'Team1 Score', 'Team2 Score', 'Winner', 'Map']]
teams

In [ ]:
team_1_df = teams[['Team1', 'Team1 Score', 'Winner', 'Map', 'Team2', 'Team2 Score']].rename(columns={
    'Team1': 'Team',
    'Team1 Score': 'Score',
    'Team2': 'Opponent',
    'Team2 Score': 'Opponent Score',
})
team_1_df['Is Winner'] = team_1_df['Team'] == team_1_df['Winner']

team_2_df = teams[['Team2', 'Team2 Score', 'Winner', 'Map', 'Team1', 'Team1 Score']].rename(columns={
    'Team2': 'Team',
    'Team2 Score': 'Score',
    'Team1': 'Opponent',
    'Team1 Score': 'Opponent Score',
})
team_2_df['Is Winner'] = team_2_df['Team'] == team_2_df['Winner']

teams = pd.concat([team_1_df, team_2_df])
teams

In [ ]:
# Dump into CSV file

prefix = datetime.utcnow().strftime('%d%m%Y_%H%M%S')

teams.to_csv(f'../data/{prefix}_teams.csv')

In [ ]:
sum_teams = teams.groupby('Team').sum('Score')
sum_teams

In [ ]:
sum_teams['Matches'] = teams.groupby('Team').size()
sum_teams

In [ ]:
sum_teams['Winrate'] = sum_teams['Is Winner'] * 100 / sum_teams['Matches']
sum_teams.sort_values('Matches', ascending=False)

In [ ]:
sum_teams[sum_teams['Matches']> 15].sort_values('Winrate', ascending=False)